In [5]:
import indexer_functions
from block_insertion import block_insertion

### Block Retrieving

In [6]:
block_height = 1194331
file = indexer_functions.block_fetching(block_height)

### Block Info Insertion

In [7]:
#create establishment to the database
conn = indexer_functions.establish_db_connection()

Connection successful!


In [8]:
#insert entries in to the established table
block_insertion(file, conn)

Transaction committed successfully.


### Block Decoding

In [6]:
txs = file['block']['data']['txs']
transac = []

#interating for each tx in the transcation records
for tx in txs:
    transac.append(indexer_functions.decoding(tx))

### Transaction Info Insertion

In [12]:
cur = conn.cursor()
query = f"SELECT block_id FROM blocks WHERE height = {block_height};"
cur.execute(query)
block_id = cur.fetchone()[0]

In [14]:
#use the transaction_insertion function to insert the info to transactions table
from transaction_insertion import transaction_insertion
transac_ids = []
for i in transac:
    transac_ids.append(transaction_insertion(i,conn,block_id))

### Message Insertion

In [9]:
import message_insertion 

message_handlers = {
    '/ibc.applications.transfer.v1.MsgTransfer':message_insertion.msg_transfer,
    '/cosmos.authz.v1beta1.MsgGrant':message_insertion.msg_grant,
    '/cosmos.staking.v1beta1.MsgDelegate': message_insertion.msg_delegate,
    '/cosmos.staking.v1beta1.MsgBeginRedelegate':message_insertion.msg_beginredelegate,
    '/cosmos.authz.v1beta1.MsgRevoke':message_insertion.msg_revoke,
    '/cosmos.distribution.v1beta1.MsgWithdrawValidatorCommission':message_insertion.,
}

In [15]:
for i in range(len(transac)):
    transac_id = transac_ids[i]
    decoded_file = transac[i]
    messages = decoded_file['tx']['body']['messages']
    for mes in messages:
        mes_type = mes['@type']
        if mes_type in message_handlers:
            insertion_function = message_handlers[mes_type]
            insertion_function(mes, conn, transac_id)
        else:
            print(f"No handler defined for message type: {mes_type}")
